## Final Project Submission

Please fill out:
* Student name: ANTONY SILA 
* Student pace: full time
* Scheduled project review date/time:
* Instructor name: MARYANN MWIKALI
* Blog post URL:


## IDENTIFYING LOWER-RISK AIRCRAFT SAFETY MAKES AND MODELS FOR FLEET DECISIONS

### Project Overview
The company is expanding into the aviation industry and plans to purchase and operate aircraft for commercial and private use. However, the company lacks experience in assessing aviation-related risks.

This project uses historical aviation accident data from the National Transportation Safety Board (NTSB) to identify aircraft types associated with lower risk. The goal is to provide data-driven insights that will help the head of the aviation division make informed aircraft purchasing decisions.

## Business Understanding

### Stakeholder
The primary stakeholder for this analysis is the **Head of the Aviation Division**, who is responsible for selecting aircraft that minimize operational and safety risks.

### Business Problem
The company needs to identify aircraft with the lowest risk profile before investing in aviation operations. Poor aircraft selection could lead to increased accidents, financial losses, and reputational damage.

### Key Business Questions
- Which aircraft categories have the lowest accident frequency?
- Which engine types are associated with fewer fatal injuries?
- How has aviation safety changed over time?
- Which aircraft manufacturers show lower accident severity?


## Data Understanding

### Data Source
The dataset was obtained from the **National Transportation Safety Board (NTSB)** and contains records of civil aviation accidents and incidents.

### Time Period
The data spans from **1962 to 2023**, covering both U.S. and international waters.

### Dataset Overview
The dataset includes information such as:
- Aircraft category
- Aircraft manufacturer
- Engine type
- Injury severity
- Aircraft damage
- Event date


In [62]:

# Import necessary libraries and load data

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
# 1️⃣ Load the dataset
df = pd.read_csv("Aviation_Data.csv", encoding="latin1", low_memory=False)

# Define keywords to search for in headers
keywords = {
    'date': 'Event_Date',
    'category': 'Aircraft_Category',
    'engine': 'Engine_Type',
    'fatal': 'Total_Fatal_Injuries',
    'damage': 'Aircraft_Damage',
    'purpose': 'Purpose_Of_Flight'
}

# Automatically find the actual column names from the file
actual_cols = {}
for key, standard_name in keywords.items():
    match = [col for col in df.columns if key.lower() in col.lower()]
    if match:
        actual_cols[match[0]] = standard_name

# Select the found columns and rename them to your preferred names
df = df[list(actual_cols.keys())].rename(columns=actual_cols)

# Show the results
df.head()

,Event_Date,Aircraft_Category,Engine_Type,Total_Fatal_Injuries,Aircraft_Damage,Purpose_Of_Flight
0,1948-10-24,NaN,1.0,2.0,Destroyed,Personal
1,1962-07-19,NaN,1.0,4.0,Destroyed,Personal
2,1974-08-30,NaN,1.0,3.0,Destroyed,Personal
3,1977-06-19,NaN,1.0,2.0,Destroyed,Personal
4,1979-08-02,NaN,NaN,1.0,Destroyed,Personal


## Data Preparation
1.cleaning column names
2.handling missing values
3. selecting relevant columns

In [63]:
# Check dataset structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Event_Date            88889 non-null  object 
 1   Aircraft_Category     32287 non-null  object 
 2   Engine_Type           82805 non-null  float64
 3   Total_Fatal_Injuries  77488 non-null  float64
 4   Aircraft_Damage       85695 non-null  object 
 5   Purpose_Of_Flight     82697 non-null  object 
dtypes: float64(2), object(4)
memory usage: 4.1+ MB


In [64]:
# Summary statistics for numerical columns
df.describe()

,Engine_Type,Total_Fatal_Injuries
count,82805.000000,77488.000000
mean,1.146585,0.647855
std,0.446510,5.485960
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,8.000000,349.000000


# missing values

In [65]:
# Check for missing values
df.isna().sum().sort_values(ascending=False)

Aircraft_Category       58061
Total_Fatal_Injuries    12860
Purpose_Of_Flight        7651
Engine_Type              7543
Aircraft_Damage          4653
Event_Date               1459
dtype: int64

In [69]:
# Drop rows missing critical info
df = df.dropna(subset=['Aircraft_Category', 'Engine_Type'])

# Fill missing fatal injuries with 0
df['Total_Fatal_Injuries'] = df['Total_Fatal_Injuries'].fillna(0)

# Optional: fill Aircraft_Damage missing values with "Unknown"
df['Aircraft_Damage'] = df['Aircraft_Damage'].fillna('Unknown')


# column selection

In [55]:
print(df.columns.tolist())


['Event_Date', 'Aircraft_Category', 'Engine_Type', 'Total_Fatal_Injuries', 'Aircraft_Damage', 'Purpose_Of_Flight']


In [56]:
df = df[relevant_columns]

# Check the selection
df.head()


,Event_Date,Aircraft_Category,Engine_Type,Total_Fatal_Injuries,Aircraft_Damage,Purpose_Of_Flight
0,1948-10-24,NaN,1.0,2.0,Destroyed,Personal
1,1962-07-19,NaN,1.0,4.0,Destroyed,Personal
2,1974-08-30,NaN,1.0,3.0,Destroyed,Personal
3,1977-06-19,NaN,1.0,2.0,Destroyed,Personal
4,1979-08-02,NaN,NaN,1.0,Destroyed,Personal


In [61]:
# Check cleaned column names
list(df.columns)


['Event_Date',
 'Aircraft_Category',
 'Engine_Type',
 'Total_Fatal_Injuries',
 'Aircraft_Damage',
 'Purpose_Of_Flight']

# date conversion

In [66]:
# Convert Event_Date to datetime
df['Event_Date'] = pd.to_datetime(df['Event_Date'], errors='coerce')

# Create a Year column for trend analysis
df['Event_Year'] = df['Event_Date'].dt.year

# Check conversion
df[['Event_Date', 'Event_Year']].head()


,Event_Date,Event_Year
0,1948-10-24,1948.0
1,1962-07-19,1962.0
2,1974-08-30,1974.0
3,1977-06-19,1977.0
4,1979-08-02,1979.0


In [71]:
# Final check
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 28830 entries, 5 to 90345
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Event_Date            28830 non-null  datetime64[ns]
 1   Aircraft_Category     28830 non-null  object        
 2   Engine_Type           28830 non-null  float64       
 3   Total_Fatal_Injuries  28830 non-null  float64       
 4   Aircraft_Damage       28830 non-null  object        
 5   Purpose_Of_Flight     26537 non-null  object        
 6   Event_Year            28830 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 1.8+ MB


,Event_Date,Aircraft_Category,Engine_Type,Total_Fatal_Injuries,Aircraft_Damage,Purpose_Of_Flight,Event_Year
5,1979-09-17,Airplane,2.0,0.0,Substantial,NaN,1979.0
7,1982-01-01,Airplane,1.0,0.0,Substantial,Personal,1982.0
8,1982-01-01,Airplane,2.0,0.0,Substantial,Business,1982.0
12,1982-01-02,Airplane,1.0,0.0,Destroyed,Personal,1982.0
13,1982-01-02,Airplane,1.0,1.0,Destroyed,Personal,1982.0


## finalcheck of cleaned dataset

In [72]:

# 1️⃣ Convert Engine_Type to categorical (example mapping)
engine_mapping = {
    1.0: 'Single Engine',
    2.0: 'Multi Engine',
    3.0: 'Jet',
    4.0: 'Rotorcraft',
    5.0: 'Other'
}
df['Engine_Type'] = df['Engine_Type'].map(engine_mapping)

# 2️⃣ Fill missing Purpose_Of_Flight
df['Purpose_Of_Flight'] = df['Purpose_Of_Flight'].fillna('Unknown')

# 3️⃣ Convert Event_Year to integer
df['Event_Year'] = df['Event_Year'].astype(int)

# 4️⃣ Standardize categorical columns
categorical_cols = ['Aircraft_Category', 'Engine_Type', 'Aircraft_Damage', 'Purpose_Of_Flight']
for col in categorical_cols:
    df[col] = df[col].str.strip().str.title()

# 5️⃣ Verify clean dataset
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 28830 entries, 5 to 90345
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Event_Date            28830 non-null  datetime64[ns]
 1   Aircraft_Category     28830 non-null  object        
 2   Engine_Type           28319 non-null  object        
 3   Total_Fatal_Injuries  28830 non-null  float64       
 4   Aircraft_Damage       28830 non-null  object        
 5   Purpose_Of_Flight     28830 non-null  object        
 6   Event_Year            28830 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), object(4)
memory usage: 1.6+ MB


,Event_Date,Aircraft_Category,Engine_Type,Total_Fatal_Injuries,Aircraft_Damage,Purpose_Of_Flight,Event_Year
5,1979-09-17,Airplane,Multi Engine,0.0,Substantial,Unknown,1979
7,1982-01-01,Airplane,Single Engine,0.0,Substantial,Personal,1982
8,1982-01-01,Airplane,Multi Engine,0.0,Substantial,Business,1982
12,1982-01-02,Airplane,Single Engine,0.0,Destroyed,Personal,1982
13,1982-01-02,Airplane,Single Engine,1.0,Destroyed,Personal,1982
